In [63]:
# First we import the libraries
import numpy as np
import matplotlib.pyplot as plt

from qucumber.nn_states import PositiveWaveFunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

import torch

# set random seed on cpu but not gpu, as gpu is not used
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [65]:
# Now we get the training data
# Training data is given in the data/nY=8/ directory for each delta
# in the '...samples.csv' files.
# So the delta needs to be specified below

#-------------------- Specify delta and nY ---------------------#
delta = "1.04" 
nY = "8" # nY refers to the number of atoms in the array
# For the nY = 8 system case:
# delta is any of 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20
# For nY > 8:
# delta is any of 1.00, 1.04, 1.08, 1.12, 1.16, 1.20, 1.28
#---------------------------------------------------------------#

train_path = "data/nY="+nY+"/δ="+delta+"_samples.csv"
train_data = data.load_data(train_path)

# The training data is stored in train_data[0]
# and the dimension of the data is:
train_data[0].shape

torch.Size([10000, 8])

In [66]:
# So now we have the training data. Our goal is to train a
# Restricted Boltzman Machine using this training data. We use
# QuCumber to create an instance of an RBM. First we need to specify the 
# number of visible nodes (nv) and number of higgen nodes (nh). 
# As we have an array of 8 atoms, we have 6 inputs and so nv = 8
nv = train_data[0].shape[-1]

# Number of hidden nodes of an RBM is a hyperparameter which depends on the 
# data we are using and which can be varied to get optimal result. 
# For this problem, nh was varied from 4 to 8 and nh = 8 gave quite good results.
# Hence, we set nh = 8.
nh = 8

# Finally we create an RBM with nv visible nodes and nh  hidden nodes.
nn_state = PositiveWaveFunction(num_visible=nv, num_hidden=nh, gpu=False)

In [67]:
# Below we have more hyperparameters of the RBM model. Like the number of 
# hidden nodes, the parameters below can be varied to get optimal results.
# The following hyperparameters seemed to work quite well. Note that when
# you have different data, then different values of the hyperparameter will give
# better solutions. 
# Further description of the parameters (if you are curious) is in:
# https://qucumber.readthedocs.io/en/stable/quantum_states.html?highlight=fit#qucumber.nn_states.PositiveWaveFunction.fit

epochs = 500
pbs = 100
nbs = pbs
lr = 0.0065 
k = 10

# Now we train our RBM using the above parameters. 
# Note that the training process will take 2 - 3 minutes

nn_state.fit(
    train_data[0],
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    time=True,
)

Total time elapsed during training: 113.514 s


In [68]:
# After training is complete, we save the parameters of the trained RBM below
# as rydberg_data.pt in the output directory.
nn_state.save("output/rydberg_data.pt")
torch.load("output/rydberg_data.pt")
# Below we have the parameters  (weights and biases) of the trained RBM:

{'rbm_am': OrderedDict([('weights',
               tensor([[ 1.3587, -2.1112,  1.6800, -0.8456, -0.1849,  0.3550, -0.4630, -0.4166],
                       [ 0.1871,  0.3049, -1.0210,  1.0056, -0.7880, -0.0321,  0.4994, -0.7517],
                       [ 0.0505,  0.6470, -0.6218, -0.8277,  2.0811, -2.8360,  2.3953, -1.5319],
                       [-1.4465,  0.6002, -0.4012,  0.0931, -0.6093,  0.8346, -1.6377,  1.6565],
                       [-2.6099,  1.3747, -0.1745, -0.6722,  0.5136,  0.1548, -1.5352,  2.4759],
                       [-0.7299,  1.3422, -2.6069,  2.3921, -1.8428,  0.4468, -0.0578, -0.3527],
                       [ 1.0107, -0.8985, -0.0631,  1.0665, -1.7709,  1.4350, -0.5167, -0.5538],
                       [ 1.0418, -1.6064,  1.4373, -1.6820,  1.1698, -1.2634,  0.6325, -1.1206]],
                      dtype=torch.float64)),
              ('visible_bias',
               tensor([-0.8279, -1.0203, -0.4945, -1.4392, -0.5224, -0.7718, -0.9102, -0.9203],
               

In [69]:
# Now we have our trained RBM. Here we reconstruct the wavefunction by sampling from the RBM. 
# Let us see step by step how it works -

# The first step is to sample from our trained RBM using QuCumber.
# We then get num_samples samples which is stored in the variable 'samples'
num_samples = 5000
samples = nn_state.sample(num_samples = num_samples , k = 10)

# Now we print out the samples in 'output/reconstructedSample.txt'
sampleList = samples.tolist() # convert to list for convenience
sampleList
with open('output/reconstructedSample.txt', 'w') as fp:
    for item in sampleList:
        fp.write("%s\n" % item)

In [70]:
# From the samples we obtain the amplitudes of each of the basis states.
# Note that here we have an 8 atom array. Hencem there are 2^8 basis states
# which are: |00000000>, |00000001>, ..., |11111111>

# Now say X is any of the states from {|00000000>, |00000001>, ..., |11111111>}
# Now, the amplitude of a state X is computed by taking the
# square root of the number of states X present in 
# our produced sample in in 'output/reconstructedSample.txt'.

# Therefore the reconstructed sample is like: 
#    |psi> = amplitudeList[0] |00000000> + amplitudeList[1] |00000001> + ... + amplitudeList[255] |11111111>  
# where amplitudeList is defined below.

# The following functions below finds the amplitudes in the way described above, stores the 
# amplitudes in amplitudeList and then also 
# prints out the amplitudes in 'output/reconstructedSample.txt'

def amplitude(sample):
    return np.sqrt(sampleList.count(sample)/num_samples)

def getBinaryString(i):
    sites = nv # nv = 8
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    tempStr = getbinary(i, sites)
    return tempStr

# Python code to convert string to list character-wise
def ConvertToList(string):
    list1=[]
    list1[:0]=string
    return list1

amplitudeList = []
def getAmplitude(sample):
    for i in range (2**nv):
        binaryString = getBinaryString(i)
        strList = ConvertToList(binaryString)
        tempStr = str(amplitude(list(map(int, strList))))
        amplitudeList.append(tempStr)
    return 0

getAmplitude(sampleList)
with open('output/reconstructedStateAmplitudes.txt', 'w') as fp:
    for item in amplitudeList:
        fp.write("%s\n" % item)

In [71]:
# So now we have obtained the reconstructed wavefunction.
# Now in the 'data' directory for each delta we have 1_pt_fun and 2_pt_fn
# 1_pt_fn is the one point function and 
# 2_pt_fn is the 2 point function.

# 1_pt_fn is the average occupation of each site. Note that
# there are 8 sites as there are 8 atoms. Occupation of each site
# refers to the proportion of atoms in the up state (state 1) 
# in each state.

# Now we get average occupation of each site by finding the 
# number of atoms in the up state in each site and then
# by dividing by the number of the produced samples 
occupation = [0]*nv
for i in range(len(sampleList)):
    j = 0
    for j in range(nv):
        occupation[j] = sampleList[i][j] + occupation[j]
for i in range(nv):
    occupation[i] = occupation[i]/len(sampleList)

# Thus the average occupation per site of each site for the reconstructed state is given below
# This data can be compared to data/nY=8/δ=delta_1_pt_fn.csv for the relevant delta

# (Note: The list in data/nY=8/δ=delta_1_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the data in 'occupation' and 'data/nY=8/δ=delta_1_pt_fn.csv' are expected to be close (which they are)
occupation

[0.3956, 0.3444, 0.4142, 0.3424, 0.4216, 0.3534, 0.4012, 0.369]

In [72]:
# Now we look at the 2 point function
# 2 point function is the covariance matrix for occupations.
# It is found by 2_pt_fn = outerProdT1 - OuterProdT2

# outerProdT1 is the average of the outer product of the spin vectors with themselves
# So we first find the sum of the outerproducts of the spin vectors with themselves.
# Then we take the average
outerProdT1 = np.zeros((nv,nv))
for i in range(len(sampleList)):
    outerProdT1 = outerProdT1 + np.outer(sampleList[i], sampleList[i])
outerProdT1 = outerProdT1/len(sampleList)

# outerProd2 is simply the outer product of the occupation vector
# with itself. Note that we already found the occupation vector for the 
# one point function

outerProdT2 = np.outer(occupation,occupation)
outerProdT2

#Hence we have:
TwoPointFunc = outerProdT1 - outerProdT2

# The TwoPointFunction data for the reconstructed state is given below.
# This matrix can be compared to data/nY=8/δ=delta_2_pt_fn.csv for the relevant delta

# (Note: The matrix in data/nY=8/δ=delta_2_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the matrix in 'TwoFuncPoint' and 'data/nY=8/δ=delta_2_pt_fn.csv' are expected 
# to be nearly equal (which they are)

# Note that the matrix in data/nY=8/δ=delta_2_pt_fn.csv may be a bit misleading. 
# The matrix generated there is symmetric and this is why the values 
# below the diagonal were not computed. However the actual matrix is symmetric
TwoPointFunc

array([[ 0.23910064, -0.11004464,  0.10274248, -0.08445344,  0.07501504,
        -0.08340504,  0.10528528, -0.1239764 ],
       [-0.11004464,  0.22578864, -0.11465048,  0.10287744, -0.08159904,
         0.07268904, -0.08197328,  0.1045164 ],
       [ 0.10274248, -0.11465048,  0.24263836, -0.11622208,  0.09817328,
        -0.08257828,  0.07862296, -0.0876398 ],
       [-0.08445344,  0.10287744, -0.11622208,  0.22516224, -0.11535584,
         0.10679584, -0.08457088,  0.0808544 ],
       [ 0.07501504, -0.08159904,  0.09817328, -0.11535584,  0.24385344,
        -0.11979344,  0.10705408, -0.0881704 ],
       [-0.08340504,  0.07268904, -0.08257828,  0.10679584, -0.11979344,
         0.22850844, -0.11418408,  0.1055954 ],
       [ 0.10528528, -0.08197328,  0.07862296, -0.08457088,  0.10705408,
        -0.11418408,  0.24023856, -0.1162428 ],
       [-0.1239764 ,  0.1045164 , -0.0876398 ,  0.0808544 , -0.0881704 ,
         0.1055954 , -0.1162428 ,  0.232839  ]])